# Using Explainers and Metrics in GraphXAI
Here, we'll walk through a quick example where we 1) generate a synthetic dataset that is ready for XAI evaluation, 2) train a GNN model, 3) explain the GNN model with two types of explainers, and 4) evaluate the output explanations from each explainer.

In [1]:
import torch
import matplotlib.pyplot as plt
from graphxai.datasets import ShapeGGen

import networkx as nx


First we load the dataset and run a quick plot of the dataset.

In [2]:
# Get dataset:
torch.set_default_device("cpu")
dataset = ShapeGGen(
            base_graph= "ba",
            verify=False,
            max_tries_verification=5,
            homophily_coef=1.0,
            seed=0,
            shape_method="house",
            sens_attribution_noise=0.3,
            num_hops=3,
            model_layers=3,
            make_explanations=True,
            variant=1,
            num_subgraphs=300,
            prob_connection=0.003,
            subgraph_size=11,
            # Features=
            class_sep=15.,
            n_features=100,
            n_clusters_per_class=2,
            n_informative=10,
            add_sensitive_feature=False,
            attribute_sensitive_feature=False,
)


100%|███████████████████████████████████████████████████████████████████████████████| 243/243 [00:01<00:00, 242.73it/s]


In [3]:
# Train a model from scratch on the data:
from torch_geometric.nn import GINConv


class MyGNN(torch.nn.Module):
    def __init__(self,input_feat, hidden_channels, classes = 2):
        super(MyGNN, self).__init__()
        self.mlp_gin1 = torch.nn.Linear(input_feat, hidden_channels)
        self.gin1 = GINConv(self.mlp_gin1)
        self.mlp_gin2 = torch.nn.Linear(hidden_channels, classes)
        self.gin2 = GINConv(self.mlp_gin2)

    def forward(self, x, edge_index):
        # NOTE: our provided testing function assumes no softmax
        #   output from the forward call.
        x = self.gin1(x, edge_index)
        x = x.relu()
        x = self.gin2(x, edge_index)
        return x

Now we get our data and train the model. The data includes a train, val, and testing mask that are generated when you call `get_graph` on the dataset. These masks are used internally in our provided train, test, and val functions.

In [4]:
from graphxai.gnn_models.node_classification import train, test

data = dataset.get_graph(use_fixed_split=True)

model = MyGNN(dataset.n_features, 32)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, weight_decay = 0.001)
criterion = torch.nn.CrossEntropyLoss()

# Train model:
for _ in range(300):
    loss = train(model, optimizer, criterion, data)

# Final testing performance:
f1, acc, prec, rec, auprc, auroc = test(model, data, num_classes = 2, get_auc = True)

print('Test F1 score: {:.4f}'.format(f1))
print('Test AUROC: {:.4f}'.format(auroc))


Test F1 score: 0.6491
Test AUROC: 0.9564


Now that we have a model trained, we can move to explaining the performance. We'll use two different explainers here from two different families of GNN explainers provided in `graphxai`: PGExplainer (perturbation-based, paper: https://arxiv.org/abs/2011.04573) and Integrated Gradients (gradient-based, paper: https://arxiv.org/abs/1703.01365).

* Note that we usually want better testing performance, but we keep the graph small here for the example. Larger graphs produce better performance overall.

In [5]:
from torch_geometric.data import Data
from torch_geometric.utils import k_hop_subgraph
from graphxai.explainers import PGExplainer,GNNExplainer, IntegratedGradExplainer, GradExplainer, GuidedBP,RandomExplainer

# Embedding layer name is final GNN embedding layer in the model
# pgex = PGExplainer(model, emb_layer_name = 'gin2', max_epochs = 10, lr = 0.1)
# gnnex = GNNExplainer(model)
# gradex = GradExplainer(model, criterion=criterion)
# guidex= GuidedBP(model)
# randex= RandomExplainer(model)
# # Required to first train PGExplainer on the dataset:
# # Feed in entire data, the internal model uses the data's train mask
# pgex.train_explanation_model(data)

# # No training with Integrated Gradients, just run the model:
# igex = IntegratedGradExplainer(model, criterion=criterion)

# # Sample a random node for visualization. Also returns a ground truth explanation:
# node_idx, gt_exp = dataset.choose_node(split = 'test')
# print(node_idx)

    
# # Get explanations from both IG and PGEx:
# pgex_exp = pgex.get_explanation_node(node_idx = node_idx,num_hops=3, x = data.x, edge_index = data.edge_index)
# gnnex_exp = gnnex.get_explanation_node(node_idx = node_idx,num_hops=3, x = data.x, edge_index = data.edge_index)
# ig_exp = igex.get_explanation_node(node_idx = node_idx,num_hops=3, x = data.x, edge_index = data.edge_index, y = data.y)
# grad_exp =gradex.get_explanation_node(node_idx = node_idx,num_hops=3, x = data.x, edge_index = data.edge_index, y = data.y)
# guid_exp=guidex.get_explanation_node(node_idx = node_idx, x = data.x, edge_index = data.edge_index, y = data.y)
# rand_exp=randex.get_explanation_node(node_idx = node_idx,num_hops=3, x = data.x, edge_index = data.edge_index)

#fig, ax = plt.subplots(1,1, figsize = (10, 8))

# Ground-truth explanations always provided as a list. In ShapeGGen, we use the first
#   element since it produces unique explanations. 
#gt_exp[0].visualize_node(num_hops = 3, graph_data = data, ax = ax[0])
#pgex_exp.visualize_node(num_hops = 3, graph_data = data, ax = ax[1])
#ig_exp.visualize_node(num_hops = 3, graph_data = data, ax = ax[2])


In [6]:

from pygod.metric import eval_roc_auc
# from SuspiciousGCN import Both as SGCN
def find_indices(lst, condition):
    return [i for i, elem in enumerate(lst) if condition(elem)]

index_anomaly=find_indices(dataset.y,lambda e:e==1)
print(len(index_anomaly))
index_normal=find_indices(dataset.y,lambda e:e==0)
index_anomaly_train=index_anomaly[0:round(len(index_anomaly)/10)]

index_normal_train=index_normal[0:round(len(index_normal)/10)]
sample_s =torch.LongTensor(index_anomaly_train)
#print(len(sample_s))
sample_n =torch.LongTensor(index_normal)

# models = SGCN(Sample_n=sample_n, Sample_s= sample_s,epoch=100,hid_dim=20,dropout=0.5,alpha=0.5,batch_size=0,lr=0.01)
# models.fit(data)
# Rn,x_n,s_n,txn,tsn = models.decision_function(data)
# print(x_n)
# modeln = SGCN(Sample_n=sample_s, Sample_s= sample_n,epoch=100,hid_dim=20,dropout=0.5,alpha=0.5,batch_size=0,lr=0.01)
# modeln.fit(data)
# Rs,x_s,s_s,txs,tss = modeln.decision_function(data)
# print(tss)
# #txn=(txn-txn.min())/(txn.max()-txn.min())
# #tsn=(tsn-tsn.min())/(tsn.max()-tsn.min())
# #txs=(txs-txs.min())/(txs.max()-txs.min())
# #tss=(tss-tss.min())/(tss.max()-tss.min())
# #print(tss)
# x_n=(x_n-x_n.min())/(x_n.max()-x_n.min())
# s_n=(s_n-x_s.min())/(s_n.max()-s_n.min())
# x_s=(x_s-x_s.min())/(x_s.max()-x_s.min())
# s_s=(s_s-s_s.min())/(s_s.max()-s_s.min())
# tsn=tsn.detach().cpu().numpy()
# tss=tss.detach().cpu().numpy()

251


In [7]:
# x_n.shape


In [8]:
# from sklearn.cluster import AgglomerativeClustering,KMeans,OPTICS,SpectralClustering,DBSCAN
# from matplotlib import pyplot
# import numpy as np
# import umap

# array=np.column_stack((x_n.detach().cpu().numpy(),s_n.detach().cpu().numpy()))

# #array=np.concatenate([txn.detach().cpu().numpy()],axis=1)
# #print(array.shape)

# # define the model
# reducer = umap.UMAP()
# embedding = reducer.fit_transform(array)
# #print(embedding)
# model =DBSCAN(eps=1.0,min_samples=5)
# # fit model and predict clusters
# #yhat = model.fit_predict(array)
# yhat = model.fit_predict(embedding)
# # retrieve unique clusters
# clusters = np.unique(yhat)
# # create scatter plot for samples from each cluster

# for cluster in clusters:
#     # get row indexes for samples with this cluster
#     row_ix = np.where(yhat == cluster)
#     #print(row_ix)
#     # create scatter of these samples
#     pyplot.scatter(embedding[row_ix, 0], embedding[row_ix, 1],alpha=0.5)
# pyplot.show()
# pyplot.scatter(embedding[index_anomaly, 0], embedding[index_anomaly, 1],c="green")
# pyplot.scatter(embedding[index_normal, 0], embedding[index_normal, 1],c="red",alpha=0.05)
# pyplot.show()

# scoren=np.zeros(len(Rn))

# for cluster in clusters:
#     score=0
#     # get row indexes for samples with this cluster
#     row_ix = np.where(yhat == cluster)
#     s=Rn[row_ix].mean()
#     for i in row_ix:
#         scoren[i]=score-len(i)
#     #print(row_ix)
#     # create scatter of these samples
#     pyplot.scatter(Rn[row_ix], Rs[row_ix])
# pyplot.show()
# pyplot.scatter(Rn[index_anomaly], Rs[index_anomaly],c="green")
# pyplot.scatter(Rn[index_normal], Rs[index_normal],c="red")

# pyplot.show()

# auc_score = eval_roc_auc(data.y.detach().cpu(), scoren)
# print(auc_score)
# #print(row_ix)
# #print(index_anomaly)
# #print(txn.detach().cpu().numpy())
# #array=np.column_stack((txn.detach().cpu().numpy(),tsn))


# #-----------------------------------------------------------------------------------------------------------------------
# array=np.column_stack((x_s.detach().cpu().numpy(),s_s.detach().cpu().numpy()))
# #array=np.column_stack((txs.detach().cpu().numpy(),tss))

# array=np.concatenate([txs.detach().cpu().numpy()],axis=1)
# print(array.shape)

# # define the model
# reducer = umap.UMAP()
# embedding = reducer.fit_transform(array)
# #print(embedding)
# model =DBSCAN(eps=0.5,min_samples=2)
# # fit model and predict clusters
# #yhat = model.fit_predict(array)
# yhat = model.fit_predict(embedding)
# # retrieve unique clusters
# clusters = np.unique(yhat)
# # create scatter plot for samples from each cluster

# for cluster in clusters:
#     # get row indexes for samples with this cluster
#     row_ix = np.where(yhat == cluster)
#     #print(row_ix)
#     # create scatter of these samples
#     pyplot.scatter(embedding[row_ix, 0], embedding[row_ix, 1],alpha=0.5)
# pyplot.show()
# pyplot.scatter(embedding[index_anomaly, 0], embedding[index_anomaly, 1],c="green")
# pyplot.scatter(embedding[index_normal, 0], embedding[index_normal, 1],c="red",alpha=0.05)
# pyplot.show()

# scores=np.zeros(len(Rs))
# for cluster in clusters:
#     # get row indexes for samples with this cluster
#     row_ix = np.where(yhat == cluster)
#     s=Rs[row_ix].mean()
#     for i in row_ix:
#         scores[i]=s+len(i)
#     #print(row_ix)
#     # create scatter of these samples
#     pyplot.scatter(Rn[row_ix], Rs[row_ix])
# pyplot.show()
# pyplot.scatter(Rn[index_anomaly], Rs[index_anomaly],c="green")
# pyplot.scatter(Rn[index_normal], Rs[index_normal],c="red")

# pyplot.show()

# auc_score = eval_roc_auc(data.y.detach().cpu(), -scores)
# print(auc_score)

# auc_score = eval_roc_auc(data.y.detach().cpu(), scoren/scores)
# print(auc_score)

In [9]:
# sx=x_n-x_s
# #print(x)
# ss=s_n/s_s
# #prints_s)
# auc_score = eval_roc_auc(data.y.detach().cpu(), Rn/Rs)
# print(auc_score)
# auc_score = eval_roc_auc(data.y.detach().cpu(), Rn)
# print(auc_score)
# auc_score = eval_roc_auc(data.y.detach().cpu(), -Rs)
# print(auc_score)
# sum=(sx.sum(1)+ss.sum(1))


In [10]:
# sum.shape
# sx.shape
# ss.shape

In [11]:
# sx=torch.div(sx.t(),sum).t()
# print(sx.shape)
# ss=torch.div(ss.t(),sum).t()
# print(ss.shape)

In [12]:

# Susps=ss
# Suspx=sx

In [13]:
# subset, sub_edge_index, mapping, hard_edge_mask = k_hop_subgraph(node_idx, 3, data.edge_index, relabel_nodes=True)
# print(len(sub_edge_index[0]))

# # subSusps=np.empty(0)
# # sub_edge_index=sub_edge_index.detach().cpu().numpy()
# # st=ss.detach().cpu().numpy()
# # for i in range(0,len(sub_edge_index[0])):
# #     subSusps=np.append(subSusps,st[sub_edge_index[0][i]][sub_edge_index[1][i]]+st[sub_edge_index[1][i]][sub_edge_index[0][i]])


    
# #print((subSusps))
# print(len(rand_exp.edge_imp))
# print(len(gnnex_exp.edge_imp))
# print(len(dataset.explanations[node_idx][0].edge_imp))

In [17]:

from pygod.detector import DOMINANT,AnomalyDAE
import numpy as np

import scipy.stats as stats
from torch_geometric.utils import to_dense_adj,to_networkx,from_networkx
import networkx as nx


modeld = DOMINANT(epoch=200,hid_dim=10,dropout=0.5,alpha=0.5,batch_size=0,gpu=0)
modeld.fit(data)
#D,x,s=modeld.decision_function(data)
pred, D, prob, conf = modeld.predict(data,return_pred=True,return_score=True,return_prob=True,return_conf=True)
x=modeld.x_imp
s=modeld.a_imp
auc_score = eval_roc_auc(data.y.detach().cpu(), D)
rank=stats.rankdata(D)
#conf_ex=ExplainerConfig("phenomenon","attributes","object")
# print(auc_score)
# print(len(s.sum(1)))
# print(len(x.sum(1)))
# sum=(s.sum(1)+x.sum(1))

# x=torch.div(x.t(),sum).t()
# s=torch.div(s.t(),sum).t()
# subDoms=np.empty(0)
# subset, sub_edge_index, mapping, hard_edge_mask = k_hop_subgraph(node_idx, 3, data.edge_index, relabel_nodes=True)
# st=s.detach().cpu().numpy()
# for i in range(0,len(sub_edge_index[0])):
#     subDoms=np.append(subDoms,st[sub_edge_index[0][i]][sub_edge_index[1][i]]+st[sub_edge_index[1][i]][sub_edge_index[0][i]])

def count(gt,exp): 
    TP=0
    FP=0
    FN=0
    for i in range(0,len(gt)):
        if exp[i]:
            if gt[i]==exp[i]:
                TP+=1
            else:
                FP+=1
        else:
            if gt[i]:
                FN+=1
                
    return([TP / (TP + FP + FN + 1e-09),TP/(TP+FP+1e-09),TP/(FN+TP+1e-09)])
def Charax(data,exp,node_idx): 
    x=data.x.clone()
    ind=torch.nonzero(exp, as_tuple=False)[:, 0]
    for k in ind:
        x[node_idx][k]=0
    dataalt=Data(x=x,edge_index=data.edge_index,y=data.y)
    predalt, scorealt, probalt, confalt = modeld.predict(dataalt,return_pred=True,return_score=True,return_prob=True,return_conf=True)
    rankalt=stats.rankdata(scorealt)
    faithx=((rank[node_idx]-rankalt[node_idx])/data.x.shape[0])
    x=data.x.clone()
    tempx=data.x.clone()
    x[node_idx]=torch.zeros(x[node_idx].shape[0])
    for k in ind:
            x[node_idx][k]=tempx[node_idx][k]
    torch.cuda.empty_cache()
    dataalt=Data(x=x,edge_index=data.edge_index,y=data.y)
    predalt, scorealt, probalt, confalt = modeld.predict(dataalt,return_pred=True,return_score=True,return_prob=True,return_conf=True)
    rankalt=stats.rankdata(scorealt)
    completx=((rank[node_idx]-rankalt[node_idx])/data.x.shape[0])

    return faithx, completx
def Charas(data,edges_exp,sub,node_idx):
    G = to_networkx(data, to_undirected=True)
    sub_nodes, sub_edge_index, mapping, edge_mask = sub
    NecG=G.copy()
    torch.cuda.empty_cache()
    ind=torch.nonzero(edges_exp, as_tuple=False)[:, 0]
    for k in ind:
        fir=sub_edge_index[0,k].item()
        sec=sub_edge_index[1,k].item()
        if(NecG.has_edge(fir,sec)):
            NecG.remove_edge(fir,sec)
    for temp in range(0,data.x.shape[0]):
        NecG.add_edge(temp,temp)
    dataalt=Data(x=data.x,edge_index=from_networkx(NecG).edge_index,y=data.y)
    predalt, scorealt, probalt, confalt = modeld.predict(dataalt,return_pred=True,return_score=True,return_prob=True,return_conf=True)
    rankalt=stats.rankdata(scorealt)
    faith=((rank[node_idx]-rankalt[node_idx])/data.x.shape[0])
    CompG=G.copy()
    for sub in range(0,len(sub_edge_index[0])):
        fir=sub_edge_index[0,sub].item()
        sec=sub_edge_index[1,sub].item()
        if(CompG.has_edge(fir,sec)):
            CompG.remove_edge(fir,sec)
    for temp in range(0,data.x.shape[0]):
        CompG.add_edge(temp,temp)
    for k in ind:
        fir=sub_edge_index[0,k].item()
        sec=sub_edge_index[1,k].item()
        CompG.add_edge(fir,sec)
    dataalt=Data(x=data.x,edge_index=from_networkx(CompG).edge_index,y=data.y)
    predalt, scorealt, probalt, confalt = modeld.predict(dataalt,return_pred=True,return_score=True,return_prob=True,return_conf=True)
    rankalt=stats.rankdata(scorealt)
    complet=((rank[node_idx]-rankalt[node_idx])/data.x.shape[0])
    return faith,complet

def get_topk_mask(tensor, k, dim=0, largest=True):
    # Use torch.topk to efficiently find top-k elements and their indices
    values, indices = torch.topk(tensor, k)

    # Create a boolean mask with the same shape as the input tensor
    mask = torch.zeros_like(tensor, dtype=torch.bool)

    # Scatter elements along the specified dimension based on indices
    mask[indices]=1

    return mask
    
def meancountexpl(gt,ex,index_anomaly,data,kx,ks,edge=True,feature=True,lbl_needed=True,hops=True):
    moyfeature=np.empty(3)
    moyedge=np.empty(3)
    i=0
    for node_idx in index_anomaly:
        i+=1
        if hops:
            if lbl_needed:
                exp=ex.get_explanation_node(node_idx = node_idx,num_hops=3, x = data.x.to("cuda:0"),label=1, edge_index = data.edge_index.to("cuda:0"), y = data.y.to("cuda:0"))
            else:
                exp=ex.get_explanation_node(node_idx = node_idx,num_hops=3, x = data.x.to("cuda:0"),label=1, edge_index = data.edge_index.to("cuda:0"))
        else:
            if lbl_needed:
                exp=ex.get_explanation_node(node_idx = node_idx, x = data.x.to("cuda:0"),label=1, edge_index = data.edge_index.to("cuda:0"), y = data.y.to("cuda:0"))
            else:
                exp=ex.get_explanation_node(node_idx = node_idx, x = data.x.to("cuda:0"),label=1, edge_index = data.edge_index.to("cuda:0"))
        if feature:
            moyfeature=np.vstack([moyfeature,count(gt[node_idx][0].feature_imp,get_topk_mask(exp.feature_imp,kx))])
        if edge:
            moyedge=np.vstack([moyedge,count(gt[node_idx][0].edge_imp,get_topk_mask(exp.edge_imp,ks))])
    return moyfeature.mean(axis=0),moyedge.mean(axis=0)
def meancountexplv2(gt,ex,index_anomaly,data,k,edge=True,feature=True,lbl_needed=True,hops=True,y=True):
    moyfeature=np.empty((round(k))*5)
    moyedge=np.empty((round(k))*5)
    f=0
    for node_idx in index_anomaly:
        f+=1
        print(f/len(index_anomaly))
        if hops:
            if lbl_needed:
                exp=ex.get_explanation_node(node_idx = node_idx,num_hops=3, x = data.x.to("cuda:0"),label=data.y.to("cuda:0"), edge_index = data.edge_index.to("cuda:0"), y = data.y.to("cuda:0"))
            else:
                exp=ex.get_explanation_node(node_idx = node_idx,num_hops=3, x = data.x.to("cuda:0"), edge_index = data.edge_index.to("cuda:0"))
        else:
            if lbl_needed:
                exp=ex.get_explanation_node(node_idx = node_idx, x = data.x.to("cuda:0"), label=data.y.to("cuda:0"),edge_index = data.edge_index.to("cuda:0"), y = data.y.to("cuda:0"))
            else:
                exp=ex.get_explanation_node(node_idx = node_idx, x = data.x.to("cuda:0"), edge_index = data.edge_index.to("cuda:0"),y = data.y.to("cuda:0"))
        
        tempf=np.empty(0)

        tempe=np.empty(0)
        for i in range(0,k):
            if feature:
                tempf=np.append(tempf,count(gt[node_idx][0].feature_imp,get_topk_mask(exp.feature_imp,i)))
                tempf=np.append(tempf,Charax(data,get_topk_mask(exp.feature_imp,i),node_idx))
            if edge:
                tempe=np.append(tempe,count(gt[node_idx][0].edge_imp,get_topk_mask(exp.edge_imp,i*2)))
                sub = k_hop_subgraph(node_idx, 3,data.edge_index)
                tempe=np.append(tempe,Charas(data,get_topk_mask(exp.edge_imp,i*2),sub,node_idx))
        if feature:
            moyfeature=np.vstack([moyfeature,tempf])
        if edge:
            moyedge=np.vstack([moyedge,tempe])
    return moyfeature.mean(axis=0),moyedge.mean(axis=0)
def meancount(gt,s,x,index_anomaly,data,kx,ks):
   
    moyfeature=np.empty(3)
    moyedge=np.empty(3)
    for node_idx in index_anomaly:
        subset, sub_edge_index, mapping, hard_edge_mask = k_hop_subgraph(node_idx, 3, data.edge_index, relabel_nodes=True)
        subs=np.empty(0)
        st=s.detach().cpu().numpy()
        for i in range(0,len(sub_edge_index[0])):
            subs=np.append(subs,st[sub_edge_index[0][i]][sub_edge_index[1][i]]+st[sub_edge_index[1][i]][sub_edge_index[0][i]])
        moyfeature=np.vstack([moyfeature,count(gt[node_idx][0].feature_imp,get_topk_mask(x[node_idx],kx))])
        moyedge=np.vstack([moyedge,count(gt[node_idx][0].edge_imp,get_topk_mask(torch.from_numpy(subs),ks))])
    return moyfeature.mean(axis=0),moyedge.mean(axis=0)
def meancountGNNEx(gt,exp,index_anomaly,model,data,k,f):
    moyfeature=np.empty(k)
    moyedge=np.empty(k)
    final=[]
    tempx=[]
    tempa=[]
    for node_idx in index_anomaly:
        print(data.x.to("cuda:0"))
        print(data.edge_index.to("cuda:0"))
        print(node_idx)
        print(data.y)
        e=exp.forward(model=model,x=data.x.to("cuda:0"),edge_index=data.edge_index.to("cuda:0"),target=data.y[node_idx],index=node_idx)
        x_imp=e.node_mask
        a_imp=e.edge_mask
        subset, sub_edge_index, mapping, hard_edge_mask = k_hop_subgraph(node_idx, 3, data.edge_index, relabel_nodes=True)
        for i in range(0,k):
            moyfeature[i]=count(gt[node_idx][0].feature_imp,get_topk_mask(x_imp,kx))
            moyedge[i]=moyedge,count(gt[node_idx][0].edge_imp,get_topk_mask(a_imp,ks))
        tempx=np.vstack([tempx,moyfeature])
        tempx=np.vstack([tempa,moyedge])
    f.write(f"GNNEx2;{i};{tempx.mean(axis=0)};{tempa.mean(axis=0)}\n")
    f.close()
    return moyfeature.mean(axis=0),moyedge.mean(axis=0)

array=np.column_stack((x.detach().cpu().numpy(),s.detach().cpu().numpy()))
#exp=GNNExplainer(modeld)
#exp.connect(explainer_config=conf_ex,model_config=conf_m)
#sub_ex_exp = sub_ex.get_explanation_node(node_idx = node_idx,x = data.x.to("cuda:0"), edge_index = data.edge_index.to("cuda:0"))
#gnnex_exp = gnnex.get_explanation_node(node_idx = node_idx,num_hops=3, x = data.x.to("cuda:0"), edge_index = data.edge_index.to("cuda:0"))
#ig_exp = igex.get_explanation_node(node_idx = node_idx,num_hops=3, x = data.x.to("cuda:0"), edge_index = data.edge_index.to("cuda:0"), y = data.y.to("cuda:0"))
#grad_exp =gradex.get_explanation_node(node_idx = node_idx,num_hops=3, x = data.x.to("cuda:0"), edge_index = data.edge_index.to("cuda:0"), y = data.y.to("cuda:0"))
#guid_exp=guidex.get_explanation_node(node_idx = node_idx, x = data.x.to("cuda:0"), edge_index = data.edge_index.to("cuda:0"), y = data.y.to("cuda:0"))


In [ ]:
from torch_geometric.data import Data
from torch_geometric.utils import k_hop_subgraph
from graphxai.explainers import *

import time

#from torch_geometric.explain import GNNExplainer,DummyExplainer, ExplainerConfig, ModelConfig
from graphxai.metrics import graph_exp_faith
# Embedding layer name is final GNN embedding layer in the model
F=["house","circle","flag"]
for i in range(0,10):
    print(i)
    for form in F:
        print(form)
        dataset = ShapeGGen(
                    base_graph= "ba",
                    verify=False,
                    max_tries_verification=5,
                    homophily_coef=1.0,
                    seed=0,
                    shape_method=form,
                    sens_attribution_noise=0.3,
                    num_hops=3,
                    model_layers=3,
                    make_explanations=True,
                    variant=1,
                    num_subgraphs=300,
                    prob_connection=0.003,
                    subgraph_size=11,
                    # Features=
                    class_sep=15.,
                    n_features=100,
                    n_clusters_per_class=2,
                    n_informative=10,
                    add_sensitive_feature=False,
                    attribute_sensitive_feature=False,
        )
        data = dataset.get_graph(use_fixed_split=True)
        index_anomaly=find_indices(dataset.y,lambda e:e==1)
        index_normal=find_indices(dataset.y,lambda e:e==0)
        index_anomaly_train=index_anomaly[0:round(len(index_anomaly)/10)]

        index_normal_train=index_normal[0:round(len(index_normal)/10)]
        modeld = DOMINANT(epoch=200,hid_dim=10,dropout=0.5,alpha=0.5,batch_size=0,gpu=0)
        modeld.fit(data)
        #D,x,s=modeld.decision_function(data)
        pred, D, prob, conf = modeld.predict(data,return_pred=True,return_score=True,return_prob=True,return_conf=True)
        pgex = PGExplainer(modeld,in_channels = 10, max_epochs = 10, lr = 0.1)
        gnnex = GNNExplainer(modeld)
        gradex = GradExplainer(modeld, criterion=criterion)
        guidex= GuidedBP(modeld)
        randex= RandomExplainer(modeld)
        sub_ex = SubgraphX(modeld, reward_method = 'gnn_score', num_hops = 3)
        GLEx = GraphLIME(modeld)
        LRPEx      = GNN_LRP(modeld)
        pgmex=PGMExplainer(modeld, explain_graph=False, p_threshold=0.1)
        #exp=GNNExplainer()
        #conf_ex=ExplainerConfig("phenomenon","attributes","object")
        #conf_m=ModelConfig("binary_classification","node","raw")
       # exp.connect(explainer_config=conf_ex,model_config=conf_m)
        
        # Required to first train PGExplainer on the dataset:
        # Feed in entire data, the internal model uses the data's train mask
        #pgex.train_explanation_model(data)

        # No training with Integrated Gradients, just run the model:
        igex = IntegratedGradExplainer(modeld, criterion=criterion)

        # Sample a random node for visualization. Also returns a ground truth explanation:
        node_idx, gt_exp = dataset.choose_node(split = 'test')
        print(node_idx)


        # Get explanations from both IG and PGEx:
        #guid_exp=GLEx.get_explanation_node(node_idx = node_idx, x = data.x.to("cuda:0"), edge_index = data.edge_index.to("cuda:0"), y = data.y.to("cuda:0"))

        #guid_exp=LRPEx.get_explanation_node(node_idx = node_idx, x = data.x.to("cuda:0"), edge_index = data.edge_index.to("cuda:0"), y = data.y.to("cuda:0"))


        #guid_exp=pgmex.get_explanation_node(node_idx = node_idx, x = data.x.to("cuda:0"), edge_index = data.edge_index.to("cuda:0"), y = data.y.to("cuda:0"))

        #rand_exp=randex.get_explanation_node(node_idx = node_idx,num_hops=3, x = data.x, edge_index = data.edge_index)

        #fig, ax = plt.subplots(1,1, figsize = (10, 8))

        # Ground-truth explanations always provided as a list. In ShapeGGen, we use the first
        #   element since it produces unique explanations. 
        #gt_exp[0].visualize_node(num_hops = 3, graph_data = data, ax = ax[0])
        #pgex_exp.visualize_node(num_hops = 3, graph_data = data, ax = ax[1])
        #ig_exp.visualize_node(num_hops = 3, graph_data = data, ax = ax[2])

        f=open(f"C:/Users/ookur/Downloads/result3"+form+"v2.csv",'a')
        #meancountGNNEx(dataset.explanations,exp,index_anomaly,modeld.model,data,10,f)

        print("dummy")
        start = time.time()
        temp=meancountexplv2(dataset.explanations,randex,index_anomaly,data,10,edge=True,feature=True,lbl_needed=False)
        end = time.time()
        t=end-start
        f.write(f"Dummy;{i};")
        
        for i in range(0,50):
            f.write(f"{temp[0][i]};")
        
        for i in range(0,50):
            f.write(f"{temp[1][i]};")
        f.write(f"{t};\n")
        print("gnnex")
        print("GNNEx")
        start = time.time()
        
        temp=meancountexplv2(dataset.explanations,gnnex,index_anomaly,data,10,edge=True,feature=True,lbl_needed=False)
        end = time.time()
        t=end-start
        f.write(f"GNNEx;{i};")
        
        for i in range(0,50):
            f.write(f"{temp[0][i]};")
        
        for i in range(0,50):
            f.write(f"{temp[1][i]};")
        f.write(f"{t};\n")
        print("SubGX")
        start = time.time()
        
        temp=meancountexplv2(dataset.explanations,sub_ex,index_anomaly,data,10,edge=True,feature=False,lbl_needed=False,hops=False)
        end = time.time()
        t=end-start
        f.write(f"SubGx;{i};")
        
        #for i in range(0,30):
         #   f.write(f"{temp[0][i]};")
        
        for i in range(0,50):
            f.write(f"{temp[1][i]};")
        f.write(f"{t};\n")
        
        
        
        #f.write(f"SubGX;{i};{meancountexplv2(dataset.explanations,sub_ex,index_anomaly,data,10,edge=True,feature=False,lbl_needed=False,hops=False)}\n")
        #f.write(f"GNNEx;{i};{meancountexpl(dataset.explanations,gnnex,index_anomaly,data,10,edge=True,feature=True,lbl_needed=False)}\n")

    #     print(meancountexpl(dataset.explanations,pgex,index_anomaly,data,i,i,edge=True,feature=False,lbl_needed=False))
        f.write(f"IGEx;{i};")
        start = time.time()
        
        temp=meancountexplv2(dataset.explanations,igex,index_anomaly,data,10,edge=False,feature=True,lbl_needed=True)
        end = time.time()
        t=end-start
        for i in range(0,50):
            f.write(f"{temp[0][i]};")
        
        #for i in range(0,30):
         #   f.write(f"{temp[1][i]};")
        f.write(f"{t};\n")
        f.write(f"GradEx;{i};")
        start = time.time()
        
        temp=meancountexplv2(dataset.explanations,gradex,index_anomaly,data,10,edge=False,feature=True,lbl_needed=True)
        end = time.time()
        t=end-start
        for i in range(0,50):
            f.write(f"{temp[0][i]};")
        
        #for i in range(0,30):
         #   f.write(f"{temp[1][i]};")
        f.write(f"{t};\n")
        f.write(f"GuidedBP;{i};")
        start = time.time()
        
        temp=meancountexplv2(dataset.explanations,guidex,index_anomaly,data,10,edge=False,feature=True,lbl_needed=False,hops=False)
        end = time.time()
        t=end-start
        for i in range(0,50):
            f.write(f"{temp[0][i]};")
        
        #for i in range(0,30):
         #   f.write(f"{temp[1][i]};")
        f.write(f"{t};\n")
        
        #f.write(f"GradEx;{i};{meancountexpl(dataset.explanations,gradex,index_anomaly,data,10,edge=False,feature=True,lbl_needed=False)}\n")
        #f.write(f"GuidedBP;{i};{meancountexpl(dataset.explanations,guidex,index_anomaly,data,10,edge=False,feature=True,lbl_needed=True,hops=False)}\n")

        # print(meancount(dataset.explanations,ss,sx,index_anomaly,data,0.9,0.9))
            #f.write(f"Ours;{i};{meancount(dataset.explanations,s,x,index_anomaly,data,i,i*2)}\n")
        f.close()

0
house


100%|███████████████████████████████████████████████████████████████████████████████| 243/243 [00:01<00:00, 206.27it/s]


896
dummy
0.00398406374501992
0.00796812749003984
0.01195219123505976
0.01593625498007968
0.0199203187250996
0.02390438247011952
0.027888446215139442
0.03187250996015936
0.035856573705179286
0.0398406374501992
0.043824701195219126
0.04780876494023904
0.05179282868525897
0.055776892430278883
0.05976095617529881
0.06374501992031872
0.06772908366533864
0.07171314741035857
0.07569721115537849
0.0796812749003984
0.08366533864541832
0.08764940239043825
0.09163346613545817
0.09561752988047809
0.099601593625498
0.10358565737051793
0.10756972111553785
0.11155378486055777
0.11553784860557768
0.11952191235059761
0.12350597609561753
0.12749003984063745
0.13147410358565736
0.13545816733067728
0.1394422310756972
0.14342629482071714
0.14741035856573706
0.15139442231075698
0.1553784860557769
0.1593625498007968
0.16334661354581673
0.16733067729083664
0.17131474103585656
0.1752988047808765
0.17928286852589642
0.18326693227091634
0.18725099601593626
0.19123505976095617
0.1952191235059761
0.19920318725099

##### 

In [ ]:
import pandas as pd
circle=pd.read_csv("C:/Users/ookur/Downloads/result3circlev2.csv",sep=';',header=None)
flag=pd.read_csv("C:/Users/ookur/Downloads/result3flagv2.csv",sep=';',header=None)
house=pd.read_csv("C:/Users/ookur/Downloads/result3housev2.csv",sep=';',header=None)
print(circle)
form=["circle","flag","house"]
feature=["Dummy","IGEx","GradEx","GuidedBP","GNNEx","Ours"]
feature=["Dummy","GNNEx","GradEx"]
temp1=[]
temp2=[]
temp3=[]
for k in form:
    for f in feature:
        temp1=[]
        temp2=[]
        temp3=[]
        temp0=circle[circle[0]==f]
        for i in range(0,10):
            temp1.append(temp0[2+i*3].mean())
            temp2.append(temp0[2+i*3].std())
            temp3.append(temp0[2+i*3].std())
        for j in range(0,10):
            temp2[j]=temp1[j] - temp2[j]
            temp3[j]=temp1[j] + temp3[j]
        print(temp1)
        plt.plot(range(0,10), temp1, label=f)
        plt.fill_between(range(0,10), temp2, temp3, alpha=0.2)
        plt.legend()
        # Customize the plot (optional)
    plt.xlabel("k")
    plt.ylabel("GEA")
    plt.title("GEA@k on features for Shape"+k)
    plt.grid(True)
    plt.savefig(k+"FG")
    plt.show()
    for f in feature:
        temp1=[]
        temp2=[]
        temp3=[]
        temp0=circle[circle[0]==f]
        for i in range(0,10):
            temp1.append(temp0[2+i*3+1][3].mean())
            temp2.append(temp0[2+i*3+1][3].std())
            temp3.append(temp0[2+i*3+1][3].std())
        for j in range(0,10):
            temp2[j]=temp1[j] - temp2[j]
            temp3[j]=temp1[j] + temp3[j]
        print(temp1)
        plt.plot(range(0,10), temp1, label=f)
        plt.fill_between(range(0,10), temp2, temp3, alpha=0.2)
        plt.legend()
        # Customize the plot (optional)
    plt.xlabel("k")
    plt.xlabel("k")
    plt.ylabel("Precision")
    plt.title("Precision@k on features for Shape"+k)
    plt.grid(True)

    plt.savefig(k+"FP")
    plt.show()

    for f in feature:
        temp1=[]
        temp2=[]
        temp3=[]
        temp0=circle[circle[0]==f]
        for i in range(0,10):
            temp1.append(temp0[2+i*3+2][4].mean())
            temp2.append(temp0[2+i*3+2][4].std())
            temp3.append(temp0[2+i*3+2][4].std())
        for j in range(0,10):
            temp2[j]=temp1[j] - temp2[j]
            temp3[j]=temp1[j] + temp3[j]
        print(temp1)
        plt.plot(range(0,10), temp1, label=f)
        plt.fill_between(range(0,10), temp2, temp3, alpha=0.2)
        plt.legend()
        # Customize the plot (optional)
    plt.xlabel("k")
    plt.xlabel("k")
    plt.ylabel("Recall")
    plt.title("Recall@k on features for Shape"+k)
    plt.grid(True)

    plt.savefig(k+"FR")
    plt.show()
    edge=["Dummy","Ours","SubGX","GNNEx"]
    #edge=["SubGX"]
    for f in edge:
        temp1=[]
        temp2=[]
        temp3=[]
        temp0=circle[circle[0]==f]
        for i in range(0,10,2):
            if f=="SubGX":
                temp1.append(temp0[temp0[1]==i][3].mean())
                temp2.append(temp0[temp0[1]==i][3].std())
                temp3.append(temp0[temp0[1]==i][3].std())
            else:
                temp1.append(temp0[temp0[1]==i][5].mean())
                temp2.append(temp0[temp0[1]==i][5].std())
                temp3.append(temp0[temp0[1]==i][5].std())
        for j in range(0,5):
            temp2[j]=temp1[j] - temp2[j]
            temp3[j]=temp1[j] + temp3[j]
        print(temp1)
        plt.plot(range(0,20,4), temp1, label=f)
        plt.fill_between(range(0,20,4), temp2, temp3, alpha=0.2)
        plt.legend()
        # Customize the plot (optional)
    plt.xlabel("k")
    plt.ylabel("GEA")
    plt.title("GEA@k on edges for Shape"+k)
    plt.grid(True)

    plt.savefig(k+"EG")

    plt.show()
    for f in edge:
        temp1=[]
        temp2=[]
        temp3=[]
        temp0=circle[circle[0]==f]
        for i in range(0,10,2):
            if f=="SubGX":
                temp1.append(temp0[temp0[1]==i][4].mean())
                temp2.append(temp0[temp0[1]==i][4].std())
                temp3.append(temp0[temp0[1]==i][4].std())
            else:
                temp1.append(temp0[temp0[1]==i][6].mean())
                temp2.append(temp0[temp0[1]==i][6].std())
                temp3.append(temp0[temp0[1]==i][6].std())
        for j in range(0,5):
            temp2[j]=temp1[j] - temp2[j]
            temp3[j]=temp1[j] + temp3[j]
        print(temp1)
        plt.plot(range(0,20,4), temp1, label=f)
        plt.fill_between(range(0,20,4), temp2, temp3, alpha=0.2)
        plt.legend()
        # Customize the plot (optional)
    plt.xlabel("k")
    plt.ylabel("Precision")
    plt.title("Precision@k on edges for Shape"+k)
    plt.grid(True)

    plt.savefig(k+"EP")
    plt.show()

    for f in edge:
        temp1=[]
        temp2=[]
        temp3=[]
        temp0=circle[circle[0]==f]
        for i in range(0,10,2):
            if f=="SubGX":
                temp1.append(temp0[temp0[1]==i][5].mean())
                temp2.append(temp0[temp0[1]==i][5].std())
                temp3.append(temp0[temp0[1]==i][5].std())
            else:
                temp1.append(temp0[temp0[1]==i][7].mean())
                temp2.append(temp0[temp0[1]==i][7].std())
                temp3.append(temp0[temp0[1]==i][7].std())
        for j in range(0,5):
            temp2[j]=temp1[j] - temp2[j]
            temp3[j]=temp1[j] + temp3[j]
        print(temp1)
        plt.plot(range(0,20,4), temp1, label=f)
        plt.fill_between(range(0,20,4), temp2, temp3, alpha=0.2)
        plt.legend()
        # Customize the plot (optional)
    plt.xlabel("k")
    plt.ylabel("Recall")
    plt.title("Recall@k on edges for Shape"+k)
    plt.grid(True)

    plt.savefig(k+"ER")
    plt.show()

In [ ]:

f=open(f"C:/Users/ookur/Downloads/result3.csv",'a')
f.write(f"allo")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
CV=pd.read_csv("C:/Users/ookur/Downloads/result3circle.csv",sep=';',header=None)

In [ ]:
circle

In [ ]:
circle=pd.read_csv("C:/Users/ookur/Downloads/result3circle.csv",sep=';',header=None)

flag=pd.read_csv("C:/Users/ookur/Downloads/result3flag.csv",sep=';',header=None)

house=pd.read_csv("C:/Users/ookur/Downloads/result3house.csv",sep=';',header=None)

In [ ]:
form=["circle","flag","house"]
feature=["Dummy","IGEx","GradEx","GuidedBP","GNNEx","Ours"]
temp1=[]
temp2=[]
temp3=[]
for k in form:
    for f in feature:
        temp1=[]
        temp2=[]
        temp3=[]
        temp0=circle[circle[0]==f]
        for i in range(0,10,2):
            temp1.append(temp0[temp0[1]==i][2].mean())
            temp2.append(temp0[temp0[1]==i][2].std())
            temp3.append(temp0[temp0[1]==i][2].std())
        for j in range(0,5):
            temp2[j]=temp1[j] - temp2[j]
            temp3[j]=temp1[j] + temp3[j]
        print(temp1)
        plt.plot(range(0,10,2), temp1, label=f)
        plt.fill_between(range(0,10,2), temp2, temp3, alpha=0.2)
        plt.legend()
        # Customize the plot (optional)
    plt.xlabel("k")
    plt.ylabel("GEA")
    plt.title("GEA@k on features for Shape"+k)
    plt.grid(True)
    plt.savefig(k+"FG")
    plt.show()
    for f in feature:
        temp1=[]
        temp2=[]
        temp3=[]
        temp0=circle[circle[0]==f]
        for i in range(0,10,2):
            temp1.append(temp0[temp0[1]==i][3].mean())
            temp2.append(temp0[temp0[1]==i][3].std())
            temp3.append(temp0[temp0[1]==i][3].std())
        for j in range(0,5):
            temp2[j]=temp1[j] - temp2[j]
            temp3[j]=temp1[j] + temp3[j]
        print(temp1)
        plt.plot(range(0,10,2), temp1, label=f)
        plt.fill_between(range(0,10,2), temp2, temp3, alpha=0.2)
        plt.legend()
        # Customize the plot (optional)
    plt.xlabel("k")
    plt.xlabel("k")
    plt.ylabel("Precision")
    plt.title("Precision@k on features for Shape"+k)
    plt.grid(True)

    plt.savefig(k+"FP")
    plt.show()

    for f in feature:
        temp1=[]
        temp2=[]
        temp3=[]
        temp0=circle[circle[0]==f]
        for i in range(0,10,2):
            temp1.append(temp0[temp0[1]==i][4].mean())
            temp2.append(temp0[temp0[1]==i][4].std())
            temp3.append(temp0[temp0[1]==i][4].std())
        for j in range(0,5):
            temp2[j]=temp1[j] - temp2[j]
            temp3[j]=temp1[j] + temp3[j]
        print(temp1)
        plt.plot(range(0,10,2), temp1, label=f)
        plt.fill_between(range(0,10,2), temp2, temp3, alpha=0.2)
        plt.legend()
        # Customize the plot (optional)
    plt.xlabel("k")
    plt.xlabel("k")
    plt.ylabel("Recall")
    plt.title("Recall@k on features for Shape"+k)
    plt.grid(True)

    plt.savefig(k+"FR")
    plt.show()
    edge=["Dummy","Ours","SubGX","GNNEx"]
    #edge=["SubGX"]
    for f in edge:
        temp1=[]
        temp2=[]
        temp3=[]
        temp0=circle[circle[0]==f]
        for i in range(0,10,2):
            if f=="SubGX":
                temp1.append(temp0[temp0[1]==i][3].mean())
                temp2.append(temp0[temp0[1]==i][3].std())
                temp3.append(temp0[temp0[1]==i][3].std())
            else:
                temp1.append(temp0[temp0[1]==i][5].mean())
                temp2.append(temp0[temp0[1]==i][5].std())
                temp3.append(temp0[temp0[1]==i][5].std())
        for j in range(0,5):
            temp2[j]=temp1[j] - temp2[j]
            temp3[j]=temp1[j] + temp3[j]
        print(temp1)
        plt.plot(range(0,20,4), temp1, label=f)
        plt.fill_between(range(0,20,4), temp2, temp3, alpha=0.2)
        plt.legend()
        # Customize the plot (optional)
    plt.xlabel("k")
    plt.ylabel("GEA")
    plt.title("GEA@k on edges for Shape"+k)
    plt.grid(True)

    plt.savefig(k+"EG")

    plt.show()
    for f in edge:
        temp1=[]
        temp2=[]
        temp3=[]
        temp0=circle[circle[0]==f]
        for i in range(0,10,2):
            if f=="SubGX":
                temp1.append(temp0[temp0[1]==i][4].mean())
                temp2.append(temp0[temp0[1]==i][4].std())
                temp3.append(temp0[temp0[1]==i][4].std())
            else:
                temp1.append(temp0[temp0[1]==i][6].mean())
                temp2.append(temp0[temp0[1]==i][6].std())
                temp3.append(temp0[temp0[1]==i][6].std())
        for j in range(0,5):
            temp2[j]=temp1[j] - temp2[j]
            temp3[j]=temp1[j] + temp3[j]
        print(temp1)
        plt.plot(range(0,20,4), temp1, label=f)
        plt.fill_between(range(0,20,4), temp2, temp3, alpha=0.2)
        plt.legend()
        # Customize the plot (optional)
    plt.xlabel("k")
    plt.ylabel("Precision")
    plt.title("Precision@k on edges for Shape"+k)
    plt.grid(True)

    plt.savefig(k+"EP")
    plt.show()

    for f in edge:
        temp1=[]
        temp2=[]
        temp3=[]
        temp0=circle[circle[0]==f]
        for i in range(0,10,2):
            if f=="SubGX":
                temp1.append(temp0[temp0[1]==i][5].mean())
                temp2.append(temp0[temp0[1]==i][5].std())
                temp3.append(temp0[temp0[1]==i][5].std())
            else:
                temp1.append(temp0[temp0[1]==i][7].mean())
                temp2.append(temp0[temp0[1]==i][7].std())
                temp3.append(temp0[temp0[1]==i][7].std())
        for j in range(0,5):
            temp2[j]=temp1[j] - temp2[j]
            temp3[j]=temp1[j] + temp3[j]
        print(temp1)
        plt.plot(range(0,20,4), temp1, label=f)
        plt.fill_between(range(0,20,4), temp2, temp3, alpha=0.2)
        plt.legend()
        # Customize the plot (optional)
    plt.xlabel("k")
    plt.ylabel("Recall")
    plt.title("Recall@k on edges for Shape"+k)
    plt.grid(True)

    plt.savefig(k+"ER")
    plt.show()

In [ ]:
feature=["Dummy","IGEx","GradEx","GuidedBP","Ours"]

for f in feature:
    temp=flag[flag[0]==f]
    plt.plot(temp[1], temp[2], label=f)
    plt.legend()
    # Customize the plot (optional)
plt.xlabel("k")
plt.ylabel("GEA")
plt.title("GEA@k on features for ShapeFlag")
plt.grid(True)

plt.savefig("FFG")
plt.show()

for f in feature:
    temp=flag[flag[0]==f]
    plt.plot(temp[1], temp[3], label=f)
    plt.legend()
    # Customize the plot (optional)
plt.xlabel("k")
plt.ylabel("Precision")
plt.title("Precision@k on features for ShapeFlag")
plt.grid(True)

plt.savefig("FFP")
plt.show()

for f in feature:
    temp=flag[flag[0]==f]
    plt.plot(temp[1], temp[4], label=f)
    plt.legend()
    # Customize the plot (optional)
plt.xlabel("k")
plt.ylabel("Recall")
plt.title("Recall@k on features for ShapeFlag")
plt.grid(True)

plt.savefig("FFR")
plt.show()
edge=["Dummy","Ours","SubGX"]
for f in edge:
    temp=flag[flag[0]==f]
    if f=="SubGX":
        plt.plot(temp[1]*2, temp[3], label=f)
    else:
        plt.plot(temp[1]*2, temp[5], label=f)
    plt.legend()
    # Customize the plot (optional)
plt.xlabel("k")
plt.ylabel("GEA")
plt.title("GEA@k on edges for ShapeFlag")
plt.grid(True)

plt.savefig("FEG")
plt.show()

for f in edge:
    temp=flag[flag[0]==f]
    if f=="SubGX":
        plt.plot(temp[1]*2, temp[4], label=f)
    else:
        plt.plot(temp[1]*2, temp[6], label=f)
    plt.legend()
    # Customize the plot (optional)
plt.xlabel("k")
plt.ylabel("Precision")
plt.title("Precision@k on edges for ShapeFlag")
plt.grid(True)

plt.savefig("FEP")
plt.show()

for f in edge:
    temp=flag[flag[0]==f]
    if f=="SubGX":
        plt.plot(temp[1]*2, temp[5], label=f)
    else:
        plt.plot(temp[1]*2, temp[7], label=f)
    
    plt.legend()
    # Customize the plot (optional)
plt.xlabel("k")
plt.ylabel("Recall")
plt.title("Recall@k on edges for ShapeFlag")

plt.grid(True)

plt.savefig("FER")
plt.show()

In [ ]:
feature=["Dummy","IGEx","GradEx","GuidedBP","Ours"]

for f in feature:
    temp=house[house[0]==f]
    plt.plot(temp[1], temp[2], label=f)
    plt.legend()
    # Customize the plot (optional)
plt.xlabel("k")
plt.ylabel("GEA")
plt.title("GEA@k on features for ShapeHouse")
plt.grid(True)

plt.savefig("HFG")
plt.show()

feature=["Dummy","IGEx","GradEx","GuidedBP","Ours"]

for f in feature:
    temp=house[house[0]==f]
    plt.plot(temp[1], temp[3], label=f)
    plt.legend()
    # Customize the plot (optional)
plt.xlabel("k")
plt.ylabel("Precision")
plt.title("Precision@k on features for ShapeHouse")
plt.grid(True)

plt.savefig("HFP")
plt.show()
feature=["Dummy","IGEx","GradEx","GuidedBP","Ours"]

for f in feature:
    temp=house[house[0]==f]
    plt.plot(temp[1], temp[4], label=f)
    plt.legend()
    # Customize the plot (optional)
plt.xlabel("k")
plt.ylabel("Recall")
plt.title("Recall@k on features for ShapeHouse")
plt.grid(True)

plt.savefig("HFR")
plt.show()
edge=["Dummy","Ours","SubGX"]
#edge=["SubGX"]
for f in edge:
    temp=house[house[0]==f]
    if f=="SubGX":
        plt.plot(temp[1]*2, temp[3], label=f)
    else:
        plt.plot(temp[1]*2, temp[5], label=f)
    plt.legend()
    # Customize the plot (optional)
plt.xlabel("k")
plt.ylabel("GEA")
plt.title("GEA@k on edges for ShapeHouse")
plt.grid(True)

plt.savefig("HEG")
plt.show()

for f in edge:
    temp=house[house[0]==f]
    if f=="SubGX":
        plt.plot(temp[1]*2, temp[4], label=f)
    else:
        plt.plot(temp[1]*2, temp[6], label=f)
    plt.legend()
    # Customize the plot (optional)
plt.xlabel("k")
plt.ylabel("Precision")
plt.title("Precision@k on edges for ShapeHouse")
plt.grid(True)

plt.savefig("HEP")
plt.show()

for f in edge:
    temp=house[house[0]==f]
    if f=="SubGX":
        plt.plot(temp[1]*2, temp[5], label=f)
    else:
        plt.plot(temp[1]*2, temp[7], label=f)
    
    plt.legend()
    # Customize the plot (optional)
plt.xlabel("k")
plt.ylabel("Recall")
plt.title("Recall@k on edges for ShapeHouse")
plt.grid(True)

plt.savefig("HER")
plt.show()